# Các Tác Nhân AI Azure với Hỗ Trợ Giao Thức Ngữ Cảnh Mô Hình (MCP)

Notebook này minh họa cách sử dụng các Tác Nhân AI Azure với các công cụ Giao Thức Ngữ Cảnh Mô Hình (MCP) để tạo một tác nhân thông minh có thể tận dụng các máy chủ MCP bên ngoài nhằm nâng cao khả năng.


## Cài đặt các gói NuGet cần thiết

Đầu tiên, chúng ta cần cài đặt gói Azure AI Agents Persistent, gói này cung cấp các chức năng cốt lõi để làm việc với Azure AI Agents.


## Lợi Ích Của Xác Thực Không Dùng Khóa

Notebook này minh họa **xác thực không dùng khóa**, mang lại nhiều lợi ích như sau:
- ✅ **Không cần quản lý API keys** - Sử dụng xác thực dựa trên danh tính Azure
- ✅ **Tăng cường bảo mật** - Không lưu trữ bí mật trong mã hoặc cấu hình
- ✅ **Tự động xoay vòng thông tin xác thực** - Azure xử lý vòng đời thông tin xác thực
- ✅ **Quyền truy cập dựa trên vai trò** - Sử dụng Azure RBAC để phân quyền chi tiết

`DefaultAzureCredential` sẽ tự động sử dụng nguồn thông tin xác thực tốt nhất có sẵn:
1. Managed Identity (khi chạy trên Azure)
2. Thông tin xác thực từ Azure CLI (trong quá trình phát triển)
3. Thông tin xác thực từ Visual Studio
4. Biến môi trường (nếu được cấu hình)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Cài đặt gói Azure Identity để xác thực với các dịch vụ Azure bằng DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Nhập Các Không Gian Tên Cần Thiết

Nhập các không gian tên cần thiết cho Azure AI Agents và Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Cấu hình Azure AI Agent Client (Xác thực không cần khóa)

Thiết lập các biến cấu hình và tạo PersistentAgentsClient sử dụng **xác thực không cần khóa**:
- **projectEndpoint**: Điểm cuối dự án Azure AI Foundry
- **modelDeploymentName**: Tên của mô hình AI đã triển khai (GPT-4.1 nano)
- **mcpServerUrl**: URL của máy chủ MCP (Microsoft Learn API)
- **mcpServerLabel**: Nhãn để nhận diện máy chủ MCP
- **DefaultAzureCredential**: Sử dụng danh tính được quản lý, Azure CLI, hoặc các nguồn thông tin xác thực khác (không yêu cầu khóa API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Tạo Định Nghĩa Công Cụ MCP

Tạo một định nghĩa công cụ MCP kết nối với máy chủ MCP của Microsoft Learn. Điều này sẽ cho phép tác nhân truy cập nội dung và tài liệu của Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Tạo tác nhân AI

Tạo một tác nhân AI liên tục với mô hình và công cụ MCP được chỉ định. Tác nhân được cấu hình với:
- Mô hình GPT-4.1 nano  
- Hướng dẫn sử dụng công cụ MCP để hỗ trợ  
- Quyền truy cập vào máy chủ Microsoft Learn MCP  


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Tạo Chủ Đề và Gửi Tin Nhắn

Tạo một chuỗi hội thoại và gửi một tin nhắn cho người dùng hỏi về sự khác biệt giữa Azure OpenAI và OpenAI. Điều này sẽ kiểm tra khả năng của tác nhân trong việc sử dụng các công cụ MCP để cung cấp thông tin chính xác.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Cấu hình Tài nguyên Công cụ MCP (Không cần khóa)

Thiết lập tài nguyên công cụ MCP. Để thực hiện phương pháp không cần khóa thực sự, bạn có thể loại bỏ các tiêu đề tùy chỉnh nếu máy chủ MCP hỗ trợ xác thực dựa trên danh tính Azure. Ví dụ dưới đây minh họa cách thêm tiêu đề nếu cần, nhưng đối với các kịch bản không cần khóa, những tiêu đề này có thể không cần thiết.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Bắt đầu chạy Agent

Tạo và bắt đầu một phiên chạy để xử lý tin nhắn của người dùng. Agent sẽ sử dụng các công cụ và tài nguyên MCP đã được cấu hình để tạo phản hồi.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Giám sát Chạy và Xử lý Phê duyệt Công cụ (Không cần khóa)

Giám sát trạng thái chạy của agent và xử lý bất kỳ phê duyệt công cụ nào cần thiết. Vòng lặp này:
1. Chờ chạy hoàn tất hoặc yêu cầu hành động
2. Tự động phê duyệt các lệnh gọi công cụ MCP khi cần
3. Đối với xác thực không cần khóa, có thể không cần tiêu đề nếu máy chủ MCP hỗ trợ nhận dạng Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Hiển thị Kết quả Cuộc trò chuyện

Lấy và hiển thị tất cả các tin nhắn trong chuỗi, bao gồm cả câu hỏi của người dùng và phản hồi của nhân viên hỗ trợ. Các tin nhắn được hiển thị theo thứ tự thời gian với dấu thời gian và chỉ báo vai trò.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
